# San Francisco: Police incident peak in the early evening, much earlier than expected

## Introduction

I would like to investigate the correlation between the time of day and the number of police incidents which were recorded  during the summer of 2014 in San Francisco. I suspect that there would be more incidents during the evening than the day when it is dark.

## Grouping police incidents by time

To see if this suspicion is true and if there are any other correlations we created this Jupyter notebook. Initially we load the San Francisco incidents dataset using pandas. We need to convert the date columns into a datetime column so that we can query it. We also split the time column into an hour and minute column to help quering.

In [1]:
import pandas as pd
df = pd.read_csv('sanfrancisco_incidents_summer_2014.csv', sep=",")
df['Date'] =  pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M').dt.hour
df['Minute'] = pd.to_datetime(df['Time'], format='%H:%M').dt.minute

We also would like the find the hour in the day with the most and least number of occurrences. To do this we group the incidents by the newly created Hour column and count the incidntnum. We then find the max and min number of occurence.

In [2]:
# Group by hour
df_HOUR = df.groupby(pd.Grouper(key="Hour")).count().reset_index()
minHour = df_HOUR.loc[df_HOUR['IncidntNum'].idxmin()].Hour
maxHour = df_HOUR.loc[df_HOUR['IncidntNum'].idxmax()].Hour

print "San Francisco - min :" + str(minHour%24) + " max : " + str(maxHour%24)

San Francisco - min :4 max : 18


Next we want to show how the number of incidents change throughout the day. However I think it would make more sense to look at the hours of the day from 6:00am to 5:59am which would show the daytime first and the nighttime second. 

To do this we will rotate the hours by 6 hours, i.e. if the time is before 6 we add 24 hours to it and then sort by the hours. We also rotate the max and min times.

In [3]:
def rotateTime(x):
    return x + 24 if (x <= 5) else x;

df_HOUR['Hour'] = df_HOUR['Hour'].transform(rotateTime)
df_HOUR = df_HOUR.sort_values(by=['Hour'], ascending=True)

minHour = rotateTime(minHour);
maxHour = rotateTime(maxHour);

To produce the plots we will use the Bokeh library. We then create a line plot of the number of incidents with time. We also add an extra vertical lines at the point for the hour of min and max occurences. We also format the time axis to display for example 10:00. 

In [4]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import FuncTickFormatter, Span, ColumnDataSource, Range1d, LinearAxis
from bokeh.transform import cumsum
from math import pi

output_notebook()

p = figure(title="Total number of police incidents at each hour throughout the summer of 2014")

p.line(x=df_HOUR["Hour"],y=df_HOUR["IncidntNum"], line_width=6, line_color="Green")

SF_MIN = Span(location=minHour, dimension='height', line_color='Green', line_width=2)
SF_MAX = Span(location=maxHour, dimension='height', line_color='Green', line_width=2)
p.renderers.extend([SF_MIN, SF_MAX])

timeFormatter = FuncTickFormatter(code="""
    var s = "0" + ((tick%1)*60.0).toFixed(0);
    return (tick % 24).toFixed(0) + ":" + s.substr(s.length-2);
""")

p.yaxis.axis_label = "Number of Incidents"
p.xaxis.axis_label = "Time of Incident"
p.xaxis[0].formatter = timeFormatter

show(p)

Loading BokehJS ...

From the graph above it can be seen that in San Francisco the peak time for police incidents is 18:00 while the lowest 
number of incidents occur at 4:00. This is as I suspected that more crimes occur in the evening than during the day (which can also be seen by the sinusodal shape of the graph. However, what is unexpected is that the peak occurs so early 18:00 , when it is still light during the summer months. 

## Grouping police incidents by description
What I suspect is that different types of incidents may occur more frequently at different times of day. To begin we first get a list of all the different descriptions present in our dataset. We group our dataset by "Descript" and count the number of incidntNums in each description. We display this below to get a list.

We can see that most incidents are "Grand Theft From Locked Auto". But we can also see that there are 368 different descriptions which are perhaps too granular to be able to compare different categories.

In [5]:
dfDescriptGROUP = df.groupby(pd.Grouper(key="Descript"))[["IncidntNum"]].count()
dfDescriptGROUP = dfDescriptGROUP.sort_values(by=["IncidntNum"], ascending=False)
print dfDescriptGROUP

                                                  IncidntNum
Descript                                                    
GRAND THEFT FROM LOCKED AUTO                            3766
STOLEN AUTOMOBILE                                       1350
LOST PROPERTY                                           1202
PETTY THEFT OF PROPERTY                                 1125
WARRANT ARREST                                           980
PETTY THEFT FROM LOCKED AUTO                             955
BATTERY                                                  903
FOUND PROPERTY                                           757
GRAND THEFT FROM UNLOCKED AUTO                           681
SUSPICIOUS OCCURRENCE                                    677
FOUND PERSON                                             637
INVESTIGATIVE DETENTION                                  559
GRAND THEFT OF PROPERTY                                  485
THREATS AGAINST LIFE                                     473
ENROUTE TO OUTSIDE JURIS

To be be able to compare the different descriptions we will attempt to group the descirptions into common groups. I created a list of categories and I will consider a description to fall within a category if that category name occurs within the description title. This is a pretty naive way to split the categories but it doesn't require detail knowledge of all the different descriptions. We then split the data into each category and calculate the number of incidents for each category and also the hour of max and min number of incidents. We also get the incidents per hour for each category.

However, there are many incident descriptions which don't fall within an any category. To account for this we will create another category "OTHER" which includes all descriptions which fall in no category.

In [6]:
# prepare data for our graphs - 
listOfCategories = ["ROBBERY","THEFT", "ASSAULT", "POSSESSION", "SALE", "HOMICIDE", "BATTERY", "LOST", "STOLEN" ];

data = {
    "categories": listOfCategories,
    "min":[],
    "max":[],
    "totIncidents":[],
    "incidentTypes":[],
    "incidntNum":[]
}

def extractDetails(df):
    data['incidentTypes'].append(df['Descript'].unique())
    data['totIncidents'].append(len(df))
    df_Hour = df.groupby(pd.Grouper(key="Hour")).count().reset_index()
    df_Hour['Hour'] = df_Hour['Hour'].transform(rotateTime)
    df_Hour_H = df_Hour.set_index("Hour")
    df_Hour_H = df_Hour_H.reindex(range(6, 6+24), fill_value=0)
    data['incidntNum'].append(df_Hour_H['IncidntNum'])
    data['max'].append(df_Hour.loc[df_Hour['IncidntNum'].idxmax()].Hour)
    data['min'].append(df_Hour.loc[df_Hour['IncidntNum'].idxmax()].Hour)

def isInNoCategory(x):
    return True if not any(s in x['Descript'] for s in listOfCategories) else False
 
#Extract only entries for each category then group by hour and get the hour with most incidents and least    
for c in data["categories"]:
    df_Filtered = df[df.apply(lambda x: c in x['Descript'], axis=1, result_type='reduce')]
    extractDetails(df_Filtered)

# Get all entries which don't fit in any category   
df_Filtered_Other = df[df.apply(isInNoCategory, axis=1, result_type='reduce')]
data['categories'].append("OTHER")
extractDetails(df_Filtered_Other)

Next we want to compare the number incidents (in total) for each category. To display this we calculate the proportion of incidents for each category. We then display a pie chart. There is a hover-over tooltip on each segment which displays the percentage proportion of each category.

In [7]:
from bokeh.palettes import brewer

allIncidents = float(sum(data['totIncidents']))
data['percent'] = map(lambda x: '{0:.2f}'.format(x/allIncidents*100.0) + "%", data['totIncidents'])
data['angle'] = map(lambda x: x/allIncidents*2*pi, data['totIncidents'])
data['color'] = brewer['Spectral'][len(data['categories'])]

p = figure(plot_height=350, title="Proportion of incident categories", toolbar_location=None,
        tools="hover", tooltips="@categories: @percent")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='categories', source=data)
p.axis.visible = None
p.xgrid.visible = False
p.ygrid.visible = False
show(p)

We can see that the majority of incidents actually do not fit in any of the categories that we have listed - so the the "OTHER" category accounts for 41.95%. The next most popular incident category is theft 32.20%. More serious types of incidents such as "ROBBERY", "HOMICIDE", "ASSAULT" account for quite little of the total incidents. "POSSESSION" and "SALE" refer to drug related incidents and they and "LOST" and "STOLEN" (which relate to property incidents) 

We can compare the time when the most occurences of a category of incidents occur for the different categories. We then display a horizontal bar chart for each category. We also display the average (i.e. for all categories) in a different colour (green) to act as a baseline. There is a hover-over tooltip which displays the hour when most incidents occur for a category.

In [8]:
from bokeh.layouts import row
from bokeh.models import FuncTickFormatter

data_MAX = {
     'categories' : list(data['categories']),
     'max': list(data['max']), 
     'colour' : ['blue']*len(data['max'])
    }

# Add the average
data_MAX["categories"].append('AVERAGE')
data_MAX["max"].append(maxHour)
data_MAX["colour"].append("green")

df_Categories = pd.DataFrame(data_MAX, columns=data_MAX.keys())
# rotate the time
df_Categories['max'] = df_Categories['max'].transform(rotateTime)
df_Categories['max_Time'] = map(lambda x: str(x%24).zfill(2)+":00", df_Categories['max'])

df_Categories = df_Categories.sort_values(by=['max'], ascending=True)

p = figure(title="Total number of police incidents at each hour throughout the summer of 2014", 
            y_range=df_Categories["categories"], tools="hover", tooltips="@categories: @max_Time")
p.hbar(height=0.8, y='categories', right="max", left=6, source=df_Categories, fill_color="colour")
p.xaxis.formatter = timeFormatter
p.xaxis.axis_label = "Peak time for incident categories"
show(p)


From the above graph we can see that although violent incident types which we more typically think of, such as "ASSAULT", "ROBBERY", "HOMICIDE" do occur late at night (22:00, 23:00 and 2:00am) but these incidents account for very little of the total. The majority of incidents which fall into the "OTHER" category have peaks early at 12:00. "THEFT" which accounts for a high proportion of incidents have a peak at 18:00.

Lastly we want to investigate the relative occurence of different categories throughout the day. To do so we will create a stacked area chart of the relative proportion of incident categories for each hour. 

The bokeh library does not support stacked graphs directly but they can be acheived using the "patches' but does require some work. 

In [9]:
import numpy as np
from bokeh.palettes import brewer

def  stacked(df):
    df_top = df.cumsum(axis=1)
    df_bottom = df_top.shift(axis=1).fillna(0)[::-1]
    df_stack = pd.concat([df_bottom, df_top], ignore_index=True)
    return df_stack

df_stacked =  pd.DataFrame(data['incidntNum']).transpose()
df_stacked.columns = data['categories']
df_stacked = df_stacked.div(df_stacked.sum(axis=1), axis=0)

areas = stacked(df_stacked)
x2 = np.hstack((df_stacked.index[::-1], df_stacked.index))
colors = brewer['Spectral'][areas.shape[1]]
p = figure(tooltips='@names')

stackedData = {
    'xs' : [x2]*areas.shape[1],
    'ys' : [areas[c].values for c in areas],
    'col' : colors,
    'names' : data['categories']
}
p.patches(xs='xs', ys='ys', color="col",  alpha=0.7, line_color='black', legend="names", source=stackedData)
p.yaxis.axis_label = "Proportion of Incidents"
p.xaxis.axis_label = "Time of Incident"
p.xaxis[0].formatter = timeFormatter

show(p)

From the above graph it can be seen that  "THEFT" category takes a higher proportion of the incidents between 17:00 and 22:00. It can also be observed that the violent types "ASSAULT" and "ROBBERY" have higher proportions during the early morning 2:00am to 4:00am. It is difficult to observe much else from this graph and the relative proportions of the different categories remains at least visually constant.

## Conclusion

In San Francisco during the summer of 2014 the peak time for incidents is 18:00. Violent types of incidents occur typically late at night but they account for little of the total number of incidents that the police deal with. Due to the high variety in incident types (which fall within the "OTHER" category) these occur throughout the day and shift the peak earlier. "THEFT" is major contributor to the number of incidents and occurs throughout the day peaking the early evening. 